In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
data_temp = pd.read_table('data/stundenwerte_TU_05100_19410101_20211231_hist/produkt_tu_stunde_19410101_20211231_05100.txt', sep=';')
data_temp = data_temp.drop('eor', axis=1)
data_temp.MESS_DATUM = pd.to_datetime(data_temp.MESS_DATUM.astype(str), format='%Y%m%d%H')
data_temp[data_temp == -999] = np.nan
data_temp = data_temp[data_temp.MESS_DATUM >= pd.to_datetime('2018-01-01')]

In [3]:
data_solar = pd.read_table('data/stundenwerte_ST_05100_row/produkt_st_stunde_19580101_20220531_05100.txt', sep=';')
data_solar = data_solar.drop('eor', axis=1)
data_solar.MESS_DATUM = pd.to_datetime(data_solar.MESS_DATUM.astype(str), format=('%Y%m%d%H:%M'))
data_solar[data_solar == -999] = np.nan
data_solar = data_solar[data_solar.MESS_DATUM >= pd.to_datetime('2018-01-01')]

In [ ]:
data_solar.head()

In [ ]:
import plotly.express as px
px.line(data_temp, x='MESS_DATUM', y='TT_TU')

In [ ]:
px.line(data_solar, x='MESS_DATUM', y='FG_LBERG')

In [12]:
kelvin = lambda x: x+273.5
celsius = lambda x: x-273.5
vorlauf_max = 40
aussen_min = -15
raum = 20
ny = 0.5

vorlauftemp = lambda t: - (vorlauf_max - 20) / (20-aussen_min) *t + vorlauf_max + aussen_min * (vorlauf_max - 20) / (20-aussen_min)
eta = lambda x: np.maximum(0, np.minimum(10, ny * kelvin(vorlauftemp(x)) / (kelvin(vorlauftemp(x))-kelvin(x))))

data_temp['eta_hp'] = eta(data_temp.TT_TU)
data_temp['vorlauf_temp'] = vorlauftemp(data_temp.TT_TU)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = data_temp.MESS_DATUM, y=  data_temp.eta_hp, name = 'Wirkungsgrad'), secondary_y=False)
fig.add_trace(go.Scatter(x = data_temp.MESS_DATUM, y=  data_temp.vorlauf_temp, name = 'Vorlauftemp'), secondary_y=True)
fig.add_trace(go.Scatter(x = data_temp.MESS_DATUM, y = data_temp.TT_TU, name = 'Außentemp'), secondary_y=True)

fig.show()

Berechnung von benötigter Energie (optimiert)

Finde minimum -> Optimierungsproblem
Beschränkungen:
- Kapazität der Wärmepumpe (Wh ist Joule / s) -> wie viel Joule kann Maximal über Zeitraum "erbracht / verarbeitet" werden? -> vereinfachte Annahme: unendliche Kapazität (kann Wasser unendlich schnell auf gewünschte Temperatur bringen)
- Wie viel Energie wird benötigt, um das Wasser warm zu halten (in abh. von sich ändernder Außentemperatur)
- Wann sollte also am besten angefangen werden, das Wasser zu erhitzen, mit wie viel Watt?
- Wie viel Wärmeenergie wird benötigt über den Lauf der Zeit? -> In Abh. von Vorlauftemperatur
- Was ist die maximale Vorlauftemperatur über Zeitraum?
- Was sind Lokale Minima in der Funktion? -> nur bis zum nächsten Maxima schauen

In [18]:
timeframe_h = 24 #hours
timeframe_s = timeframe_h * 3600 #seconds


In [26]:
#get local minima and maxima in function
from scipy.signal import argrelextrema

check_interval = 18
data_temp['local_min'] = data_temp.iloc[argrelextrema(data_temp.TT_TU.values, np.less_equal, order=check_interval)[0]]['TT_TU']
data_temp['local_max'] = data_temp.iloc[argrelextrema(data_temp.TT_TU.values, np.greater_equal, order=check_interval)[0]]['TT_TU']
data_temp['local_max_vorlauf'] = data_temp.iloc[argrelextrema(data_temp.vorlauf_temp.values, np.greater_equal, order=check_interval)[0]]['vorlauf_temp']

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = data_temp.MESS_DATUM, y = data_temp.TT_TU), secondary_y = False)
fig.add_trace(go.Scatter(x = data_temp.MESS_DATUM, y = data_temp.local_min, mode='markers'), secondary_y = False)
fig.add_trace(go.Scatter(x = data_temp.MESS_DATUM, y = data_temp.local_max, mode='markers'), secondary_y = False)
fig.add_trace(go.Scatter(x = data_temp.MESS_DATUM, y = data_temp.local_max_vorlauf, mode='markers'), secondary_y= True)
fig.add_trace(go.Scatter(x = data_temp.MESS_DATUM, y = data_temp.vorlauf_temp, mode='lines'), secondary_y= True)
fig.show()

In [97]:
#take Newton cooling data

#20 and 60 after some DIN-Norms
t_env = 20 + 273.15
t_0 = 60 + 273.15
#coef_h = 0.000625

#specifications of water storage (taken from real example)
hp_power = 72 #Watts
h2o_energy = 4200 #Joule / (kilogram * Kelvin)
hp_volume = 252 #Liter = kilogram


def newton_cooling(t):
    return t_env + (t_0 - t_env) * np.exp(-coef_h * t) # = temperature at time t

def newton_cooling_diff(t):
    return coef_h * (t_env - newton_cooling(t)) # = temperature loss rate at time t

def newton_cooling_t_0(temp_t, t=7*3600):
    return (temp_t - t_env) * np.exp(coef_h * t) + t_env #returns t_0 for fix temperature after time t

def newton_cooling_t(temp_t):
    return -np.log((temp_t - t_env) / (t_0 - t_env)) / coef_h #returns how much time it takes for t_0 to cool down to wished temperature

def hp_power_temp(temp):
    return  -h2o_energy * hp_volume * newton_cooling_diff(newton_cooling_t(temp))#returns how much power in Watt is needed to keep temperature at temp level

def get_d_t():
    return hp_power / (h2o_energy * hp_volume)

def get_coef_h(d_t):
    return d_t / np.abs(t_env - t_0)


d_t = get_d_t()
coef_h = get_coef_h(d_t)

In [92]:
from scipy.integrate import trapz

integral = lambda x, y: trapz(y, x)
integral_avg = lambda x, y: integral(x, y) / (x.shape[0] - 1)


overheat = lambda time, vorlauftemp, eta_t_0: (newton_cooling_t_0(kelvin(vorlauftemp), t = time*3600) - 273.15 - vorlauftemp) * h2o_energy * hp_volume / eta_t_0
keepheat = lambda time, vorlauftemp, eta_avg: hp_power_temp(kelvin(vorlauftemp))*3600*time / eta_avg

In [76]:
exmpl = data_temp[(data_temp.MESS_DATUM >= pd.to_datetime('2021-02-12 13:00:00')) & (data_temp.MESS_DATUM <= pd.to_datetime('2021-02-13 07:00:00'))].reset_index(drop=True)
ex_x = exmpl.index
ex_y = exmpl.eta_hp
int_avg = integral_avg(ex_x, ex_y)

In [ ]:
exmpl

In [ ]:
overheat(18, exmpl.vorlauf_temp.values[-1], exmpl.eta_hp.values[0])

In [ ]:
keepheat(18, exmpl.vorlauf_temp.values[-1], int_avg)